# **Movie Recomendation system**

This project is a movie recommendation system that uses machine learning techniques to suggest movies to users based on their input. 

The user is prompted to input a movie name. The system uses difflib (a library for comparing sequences) to find the closest matching movie title from the dataset.

Once a matching movie is found, the system retrieves its details and calculates its similarity with other movies in the dataset.
    
The system then ranks the movies based on their similarity scores across different features (genres, cast, release date, and other factors) and recommends the top 10 movies that are most similar to the input movie.

## Importing the packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

## Data Preparation

In [2]:
data=pd.read_csv("Movies_data.csv")

### Details of File

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   index                 4809 non-null   object
 1   budget                4809 non-null   object
 2   genres                4781 non-null   object
 3   homepage              1718 non-null   object
 4   id                    4809 non-null   object
 5   keywords              4397 non-null   object
 6   original_language     4809 non-null   object
 7   original_title        4809 non-null   object
 8   overview              4806 non-null   object
 9   popularity            4809 non-null   object
 10  production_companies  4809 non-null   object
 11  production_countries  4809 non-null   object
 12  release_date          4808 non-null   object
 13  revenue               4809 non-null   object
 14  runtime               4807 non-null   object
 15  spoken_languages      4809 non-null   

In [4]:
data.shape

(4809, 24)

### Getting the req_data from data set

In [5]:
req_data=data.iloc[:,[2,5,8,9,12,14,15,19,21,23]]

### Converting all data to string format and filling the null values

In [17]:
for i in req_data.keys():
    req_data.loc[:, i] = req_data[i].astype("string")
    req_data.loc[:, i] = req_data[i].fillna("")

In [18]:
req_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            4809 non-null   string
 1   keywords          4809 non-null   string
 2   overview          4809 non-null   string
 3   popularity        4809 non-null   string
 4   release_date      4809 non-null   string
 5   runtime           4809 non-null   string
 6   spoken_languages  4809 non-null   string
 7   vote_average      4809 non-null   string
 8   cast              4809 non-null   string
 9   director          4809 non-null   string
dtypes: string(10)
memory usage: 375.8 KB


### Feature Extraction

In [19]:
#movie recomdations based on genres,release_date,cast,director,and remain
string=req_data['keywords']+" "+req_data['director']+" "+req_data['overview']+" "+req_data['popularity']+" "+req_data['runtime']+" "+req_data['spoken_languages']+" "+req_data['vote_average']



*   Creation of individual normalised vector for primary factors
*   similarly common normalised vector for secondary factors



In [20]:
vector1=TfidfVectorizer().fit_transform(req_data["genres"])
vector2=TfidfVectorizer().fit_transform(req_data["release_date"]+" "+req_data["popularity"])
vector3=TfidfVectorizer().fit_transform(req_data["cast"])
vector4=TfidfVectorizer().fit_transform(string)

## Similarity Calculations

In [21]:
similar_genres=cosine_similarity(vector1)
similar_release_date=cosine_similarity(vector2)
similar_cast=cosine_similarity(vector3)
similar_other_factors=cosine_similarity(vector4)

## User Input

In [22]:
while (True):
  movie=input("Enter Movie Name:")
  movie_name=difflib.get_close_matches(movie,data["original_title"])
  if(len(movie_name)==0):
    print("No recomendations found")
    print("Try with other name")
  else:
    break
val=data[data.title==movie_name[0]]["index"].values[0]
movie=data.loc[[int(val)],["title"]].values[0][0]
genres=data.loc[[int(val)],["genres"]].values[0][0]
cast=data.loc[[int(val)],["cast"]].values[0][0]
release_date=data.loc[[int(val)],["release_date"]].values[0][0]
print("Movie Name:",movie)
print("genres:",genres)
print("cast:",cast)
print("release date:",release_date)

Enter Movie Name: blue book


No recomendations found
Try with other name


Enter Movie Name: avatar


Movie Name: Avatar
genres: Action Adventure Fantasy Science Fiction
cast: Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez
release date: 10-12-2009


## Recomended Movies

In [23]:
genres_score=list(enumerate(similar_genres[int(val)]))
cast_score=list(enumerate(similar_cast[int(val)]))
release_score=list(enumerate(similar_release_date[int(val)]))
others_score=list(enumerate(similar_other_factors[int(val)]))

### Genres

In [24]:
count=0
recomanded_movies=[]
sorted_recomandations=sorted(genres_score,key=lambda x:x[1],reverse=True)
for i in sorted_recomandations:
  index=i[0]
  recomanded_movies.append(data[data.index==index]['title'].values[0])
  count+=1
  if(count>(10)):
    break
for i in recomanded_movies:
  va=data[data.title==i]["homepage"].values[0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Avatar http://www.avatarmovie.com/
Superman Returns http://www.superman.com
Man of Steel http://www.manofsteel.com/
X-Men: Days of Future Past http://www.x-menmovies.com/
Jupiter Ascending http://www.jupiterascending.com
The Wolverine http://www.thewolverinemovie.com
Superman
Superman II
Beastmaster 2: Through the Portal of Time
Teenage Mutant Ninja Turtles http://www.teenagemutantninjaturtlesmovie.com
Mystery Men


### Release Date and Popularity

In [25]:
count=0
recomanded_movies=[]
sorted_recomandations=sorted(cast_score,key=lambda x:x[1],reverse=True)
for i in sorted_recomandations:
  index=i[0]
  recomanded_movies.append(data[data.index==index]['title'].values[0])
  count+=1
  if(count>(10)):
    break
for i in recomanded_movies:
  va=data[data.title==i]["homepage"].values[0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Avatar http://www.avatarmovie.com/
Gettysburg
Out of the Furnace
Galaxy Quest
Imaginary Heroes http://www.sonypictures.com/classics/imaginary/site.html
Snow White: A Tale of Terror
The Words http://www.thewordsmovie.com/
Everest http://www.everestmovie.com/
Drumline
Get Over It
Vantage Point http://www.vantagepoint-movie.com/index.php


### Cast 

In [26]:
count=0
recomanded_movies=[]
sorted_recomandations=sorted(release_score,key=lambda x:x[1],reverse=True)
for i in sorted_recomandations:
  index=i[0]
  recomanded_movies.append(data[data.index==index]['title'].values[0])
  count+=1
  if(count>(10)):
    break
for i in recomanded_movies:
  va=data[data.title==i]["homepage"].values[0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Avatar http://www.avatarmovie.com/
A Shine of Rainbows
Life During Wartime http://www.ifcfilms.com/films/life-during-wartime-2
Chicago Overcoat
Invictus http://invictusmovie.warnerbros.com
Observe and Report
Oceans http://oceans-lefilm.com/
2012 http://www.sonypictures.com/movies/2012
Defendor
A Woman, a Gun and a Noodle Shop
Rocket Singh: Salesman of the Year http://www.yashrajfilms.com/microsites/rocketsingh/rswebsite.html


### Adds on Interest 

In [27]:
count=0
recomanded_movies=[]
sorted_recomandations=sorted(others_score,key=lambda x:x[1],reverse=True)
for i in sorted_recomandations:
  index=i[0]
  recomanded_movies.append(data[data.index==index]['title'].values[0])
  count+=1
  if(count>10):
    break
for i in recomanded_movies:
  va=data[data.title==i]["homepage"].values[0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Avatar http://www.avatarmovie.com/
Lifeforce
Moonraker http://www.mgm.com/view/movie/1292/Moonraker/
Gattaca
Gravity http://gravitymovie.warnerbros.com/
Cargo http://www.cargoderfilm.ch http://cargothemovie.com
Space Chimps http://www.spacechimpspower.com/
Apollo 18 http://apollo18movie.net/
Starship Troopers
Deep Impact
Alien https://www.facebook.com/alienanthology/
